### RAG Pipelines - Data ingestion to Vector DB Pipelines

In [1]:
import os
from langchain_community.document_loaders import PyPDFLoader, PyMuPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from pathlib import Path

/mnt/5d9f5ec4-97b1-4495-8630-aafd5bd9fb5d/rag/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
def process_all_pdfs(pdf_directory):
    """Process all PDF files in a directory"""
    all_documents = [ ]
    pdf_dir = Path (pdf_directory)
    # Find all PDF files recursively
    pdf_files = list (pdf_dir.glob ("**/*.pdf"))
    print (f"Found {len (pdf_files)} PDF files to process")
    for pdf_file in pdf_files:
        print("\nProcessing: {pdf_file.name}")
        try:
            loader = PyPDFLoader(str(pdf_file))
            documents = loader.load()
            # Add source information to metadata
            for doc in documents:
                doc.metadata['source_file'] = pdf_file.name
                doc.metadata['file_type'] = 'pdf'
            all_documents.extend(documents)
            print (f" ✓ Loaded {len (documents)} pages")
        except Exception as e:
            print (f" X Error: {e}")
    print(f"\nTotal documents loaded: {len(all_documents)}")
    return all_documents

all_pdf_documents = process_all_pdfs("../data")    

Found 3 PDF files to process

Processing: {pdf_file.name}
 ✓ Loaded 2 pages

Processing: {pdf_file.name}
 ✓ Loaded 11 pages

Processing: {pdf_file.name}
 ✓ Loaded 16 pages

Total documents loaded: 29


In [3]:
all_pdf_documents

[Document(metadata={'producer': 'Skia/PDF m127', 'creator': 'Chromium', 'creationdate': '2025-09-15T08:32:37+00:00', 'moddate': '2025-09-15T08:32:37+00:00', 'source': '../data/pdf_files/pplx_search.pdf', 'total_pages': 2, 'page': 0, 'page_label': '1', 'source_file': 'pplx_search.pdf', 'file_type': 'pdf'}, page_content="Hello! How can I assist you today?\nWhen resetting Windows 11 in a dual-boot system with Linux, the Windows reset process may\noverwrite the bootloader, which can remove or interfere with the GRUB bootloader that manages\nLinux boot. To avoid losing GRUB when resetting Windows 11, consider these key points:\nIn summary, resetting Windows 11 alone tends to remove GRUB, but if you are prepared to\nrestore GRUB using a Linux Live USB afterward, the dual boot can be recovered without data\nloss on Linux. Carefully manage BIOS boot priorities and be ready to reinstall GRUB if needed.\nIf preferred, detailed steps for restoring GRUB after a Windows reset or installation are av

### Text Into Chunks

In [4]:
def split_documents(documents, chunk_size=1000, chunk_overlap=200):
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=chunk_size,
        chunk_overlap=chunk_overlap,
        length_function=len,
        separators=["\n\n","\n", " ", ""]
    )
    split_docs = text_splitter.split_documents (documents)
    print (f"Split {len (documents)} documents into {len(split_docs)} chunks")
    # Show example of a chunk
    if split_docs:
        print("\nExample chunk: ")
        print (f"Content: {split_docs[0].page_content[:200]}...")
        print (f"Metadata: {split_docs[0].metadata}")
    return split_docs

chunks = split_documents(all_pdf_documents)
chunks

Split 29 documents into 131 chunks

Example chunk: 
Content: Hello! How can I assist you today?
When resetting Windows 11 in a dual-boot system with Linux, the Windows reset process may
overwrite the bootloader, which can remove or interfere with the GRUB bootl...
Metadata: {'producer': 'Skia/PDF m127', 'creator': 'Chromium', 'creationdate': '2025-09-15T08:32:37+00:00', 'moddate': '2025-09-15T08:32:37+00:00', 'source': '../data/pdf_files/pplx_search.pdf', 'total_pages': 2, 'page': 0, 'page_label': '1', 'source_file': 'pplx_search.pdf', 'file_type': 'pdf'}


[Document(metadata={'producer': 'Skia/PDF m127', 'creator': 'Chromium', 'creationdate': '2025-09-15T08:32:37+00:00', 'moddate': '2025-09-15T08:32:37+00:00', 'source': '../data/pdf_files/pplx_search.pdf', 'total_pages': 2, 'page': 0, 'page_label': '1', 'source_file': 'pplx_search.pdf', 'file_type': 'pdf'}, page_content='Hello! How can I assist you today?\nWhen resetting Windows 11 in a dual-boot system with Linux, the Windows reset process may\noverwrite the bootloader, which can remove or interfere with the GRUB bootloader that manages\nLinux boot. To avoid losing GRUB when resetting Windows 11, consider these key points:\nIn summary, resetting Windows 11 alone tends to remove GRUB, but if you are prepared to\nrestore GRUB using a Linux Live USB afterward, the dual boot can be recovered without data\nloss on Linux. Carefully manage BIOS boot priorities and be ready to reinstall GRUB if needed.\nIf preferred, detailed steps for restoring GRUB after a Windows reset or installation are av

### Embeddings and VectorStoreDB

In [5]:
import numpy as np
from sentence_transformers import SentenceTransformer
import chromadb
from chromadb.config import Settings
import uuid
from typing import List, Dict, Any, Tuple
from sklearn.metrics.pairwise import cosine_similarity

In [6]:
class EmbeddingManager:
    def __init__(self, model_name: str = "all-MiniLM-L6-v2"):
        """
        Initialize the embedding manager
        Args:
            model_name: Hugging Face model name for sentence embeddings
        """
        self.model_name = model_name
        self.model = None
        self._load_model()
    def _load_model(self):
        """Load the Sentence Transformer model"""
        try:
            print (f"Loading embedding model: {self.model_name}")
            self.model = SentenceTransformer(self.model_name)
            print (f"Model loaded successfully. Embedding dimension: {self.model.get_sentence_embedding_dimension()}")
        except Exception as e:
            print (f"Error loading model {self.model_name}: {e}")
            raise
    def generate_embeddings(self, texts: List[str]) -> np.ndarray:
        """
        Generate embeddings for a list of texts
        Args:
            texts: List of text strings to embed
        Returns:
            numpy array of embeddings with shape (len(texts), embedding_dim)
        """
        if not self.model:
            raise ValueError("Model not loaded")
        print (f"Generating embeddings for {len(texts)} texts...")
        embeddings = self.model.encode(texts, show_progress_bar=True)
        print (f"Generated embeddings with shape: {embeddings.shape}")
        return embeddings

embedding_manager = EmbeddingManager()
embedding_manager

Loading embedding model: all-MiniLM-L6-v2
Model loaded successfully. Embedding dimension: 384


### VectorStore

In [7]:
class VectorStore:
    """Manages document embeddings in a ChromaDB vector store"""
    def __init__(self, collection_name: str = "pdf_documents", persist_directory: str = "../data/vector_store"):
        """
        Initialize the vector store
        Args:
            collection_name: Name of the ChromaDB collection
        persist_directory: 
            Directory to persist the vector store
        """
        self.collection_name = collection_name
        self.persist_directory = persist_directory
        self.client = None
        self.collection = None
        self._initialize_store()

    def _initialize_store(self):
        """Initialize ChromaDB client and collection"""
        try:
            # Create persistent ChromaDB client
            os.makedirs(self.persist_directory, exist_ok=True)
            self.client = chromadb.PersistentClient (path=self.persist_directory)
            # Get or create collection
            self.collection = self.client.get_or_create_collection(
                name=self.collection_name,
                metadata={"description": "PDF document embeddings for RAG"}
            )
            print (f"Vector store initialized. Collection: {self.collection_name}")
            print(f"Existing documents in collection: {self.collection.count()}")
        except Exception as e:
            print(f"Error initializing vector store: {e}")
            raise
    def add_documents(self, documents: List[Any], embeddings: np.ndarray):
        """
        Add documents and their embeddings to the vector store
        Args:
        documents: List of LangChain documents
        embeddings: Corresponding embeddings for the documents
        """
        if len(documents) != len(embeddings):
            raise ValueError("Number of documents must match number of embeddings")
        print(f"Adding {len (documents)} documents to vector store...")
        # Prepare data for ChromaDB
        ids = []
        metadatas = []
        documents_text = []
        embeddings_list = []
        for i, (doc, embedding) in enumerate (zip(documents, embeddings)):
            # Generate unique ID
            doc_id = f"doc_{uuid.uuid4().hex[:8]}_{i}"
            ids.append(doc_id)
            # Prepare metadata
            metadata = dict(doc.metadata)
            metadata['doc_index'] = i
            metadata['content_length'] = len(doc.page_content)
            metadatas.append(metadata)
            # Document content
            documents_text.append(doc.page_content)

            # Embedding
            embeddings_list.append(embedding.tolist())
            # Add to collection
            try:
                self.collection.add(
                    ids=ids,
                    embeddings=embeddings_list,
                    metadatas=metadatas,
                    documents=documents_text
                )
                print (f"Successfully added {len (documents)} documents to vector store")
                print (f"Total documents in collection: {self.collection.count()}")
            except Exception as e:
                print (f"Error adding documents to vector store: {e}")
                raise

vector_store = VectorStore()
vector_store


Vector store initialized. Collection: pdf_documents
Existing documents in collection: 219


In [8]:
# chunks
# ## Convert text to embeddings
texts = [doc.page_content for doc in chunks]
# texts

## Generate the embeddings
embeddings = embedding_manager.generate_embeddings(texts)

## Store in vector DB
vector_store.add_documents(chunks, embeddings)




Generating embeddings for 131 texts...


Batches: 100%|██████████| 5/5 [00:01<00:00,  3.28it/s]


Generated embeddings with shape: (131, 384)
Adding 131 documents to vector store...
Successfully added 131 documents to vector store
Total documents in collection: 220
Successfully added 131 documents to vector store
Total documents in collection: 221
Successfully added 131 documents to vector store
Total documents in collection: 222
Successfully added 131 documents to vector store
Total documents in collection: 223
Successfully added 131 documents to vector store
Total documents in collection: 224
Successfully added 131 documents to vector store
Total documents in collection: 225
Successfully added 131 documents to vector store
Total documents in collection: 226
Successfully added 131 documents to vector store
Total documents in collection: 227
Successfully added 131 documents to vector store
Total documents in collection: 228
Successfully added 131 documents to vector store
Total documents in collection: 229
Successfully added 131 documents to vector store
Total documents in collecti

### Retrieval Pipeline

In [9]:
class RAGRetriever:
    """Handles query-based retrieval from the vector store"""

    def __init__(self, vector_store, embedding_manager):
        """
        Initialize the retriever
        Args:
            vector_store: Vector store containing document embeddings
            embedding_manager: Manager for generating query embeddings
        """
        self.vector_store = vector_store
        self.embedding_manager = embedding_manager

    def retrieve(self, query: str, top_k: int = 5, score_threshold: float = 0.0) -> List[Dict[str, Any]]:
        """
        Retrieve relevant documents for a query
        Args:
            query: The search query
            top_k: Number of top results to return
            score_threshold: Minimum similarity score threshold
        Returns:
            List of dictionaries containing retrieved documents and metadata
        """
        print(f"Retrieving documents for query: '{query}'")
        print(f"Top K: {top_k}, Score threshold: {score_threshold}")

        # Generate query embedding
        query_embedding = self.embedding_manager.generate_embeddings([query])[0]

        # Search in vector store
        try:
            results = self.vector_store.collection.query(
                query_embeddings=[query_embedding.tolist()],
                n_results=top_k
            )

            retrieved_docs = []
            if results.get("documents") and results["documents"][0]:
                documents = results["documents"][0]
                metadatas = results["metadatas"][0]
                distances = results["distances"][0]
                ids = results["ids"][0]

                for i, (doc_id, document, metadata, distance) in enumerate(
                    zip(ids, documents, metadatas, distances)
                ):
                    similarity_score = 1 - distance  # convert cosine distance → similarity
                    if similarity_score >= score_threshold:
                        retrieved_docs.append({
                            "id": doc_id,
                            "content": document,
                            "metadata": metadata,
                            "similarity_score": similarity_score,
                            "distance": distance,
                            "rank": i + 1
                        })

                print(f"Retrieved {len(retrieved_docs)} documents (after filtering).")
            else:
                print("No documents found in results.")

            return retrieved_docs

        except Exception as e:
            print(f"Error during retrieval: {e}")
            return []


rag_retriever = RAGRetriever(vector_store, embedding_manager)
rag_retriever


In [10]:
# rag_retriever.retrieve("Transformer architecture")
# print(vector_store.collection.count())
rag_retriever.retrieve("Multi head attention")



Retrieving documents for query: 'Multi head attention'
Top K: 5, Score threshold: 0.0
Generating embeddings for 1 texts...


Batches: 100%|██████████| 1/1 [00:00<00:00,  6.39it/s]

Generated embeddings with shape: (1, 384)


Retrieved 5 documents (after filtering).


[{'id': 'doc_56fb41a8_20',
  'content': 'MultiHead(Q,K,V ) = Concat(head1,..., headh)WO\nwhere headi = Attention(QWQ\ni ,KW K\ni ,VW V\ni )\nWhere the projections are parameter matricesWQ\ni ∈Rdmodel×dk , WK\ni ∈Rdmodel×dk , WV\ni ∈Rdmodel×dv\nand WO ∈Rhdv×dmodel .\nIn this work we employ h = 8 parallel attention layers, or heads. For each of these we use\ndk = dv = dmodel/h= 64. Due to the reduced dimension of each head, the total computational cost\nis similar to that of single-head attention with full dimensionality.\n3.2.3 Applications of Attention in our Model\nThe Transformer uses multi-head attention in three different ways:\n• In "encoder-decoder attention" layers, the queries come from the previous decoder layer,\nand the memory keys and values come from the output of the encoder. This allows every\nposition in the decoder to attend over all positions in the input sequence. This mimics the\ntypical encoder-decoder attention mechanisms in sequence-to-sequence models such as\n[3

### Integrating VectorDB Context with LLM

In [ ]:
from langchain_groq import ChatGroq
import os
from dotenv import load_dotenv
from langchain.messages import SystemMessage, HumanMessage

load_dotenv()

groq_api_key = os.getenv("GROQ_API_KEY")

llm = ChatGroq(groq_api_key=groq_api_key, model_name="openai/gpt-oss-20b", temperature=0.1, max_tokens=1024)

## 2. Simple RAG function: retrieve context + generate response
def rag_simple (query, retriever, llm, top_k=3):
    ## retriever the context
    results = retriever.retrieve(query, top_k=top_k)
    context = "\n\n".join([doc['content'] for doc in results]) if results else ""
    if not context:
        return "No relevant context found to answer the question."
    ## generate the answwer using GROQ LLM
    prompt = f"""Use the following context to answer the question concisely.
                    Context: {context}
                    Question: {query}
                    Answer:
                """
    response = llm.invoke([prompt.format(context=context, query=query)])
    return response.content


# def rag_simple(query, retriever, llm, top_k=3):
#     # 1. Retrieve relevant context
#     results = retriever.retrieve(query, top_k=top_k)
#     context = "\n\n".join([doc['content'] for doc in results]) if results else ""

#     if not context:
#         return "No relevant context found to answer the question."

#     # 2. Build the prompt
#     prompt = f"""
#         You are an AI assistant answering based on the provided context.

#         Context:
#         {context}

#         Question: {query}

#         Answer concisely and clearly:
#     """

#     # 3. Generate response using Groq LLM
#     response = llm.invoke([HumanMessage(content=prompt)])
#     return response.content




In [41]:
ans = rag_simple("What is attention mechanism", rag_retriever, llm)
ans

Retrieving documents for query: 'What is attention mechanism'
Top K: 3, Score threshold: 0.0
Generating embeddings for 1 texts...


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches: 100%|██████████| 1/1 [00:00<00:00,  7.66it/s]

Generated embeddings with shape: (1, 384)
Retrieved 0 documents (after filtering).


'No relevant context found to answer the question.'

### Advances RAG

In [48]:
def rag_advanced (query, retriever, llm, top_k=5, min_score=0.2, return_context=False):
    """
    RAG pipeline with extra features:
    - Returns answer, sources, confidence score, and optionally full context.
    """
    results = retriever.retrieve (query, top_k=top_k, score_threshold=min_score)
    if not results:
        return {'answer': 'No relevant context found.', 'sources': [], 'confidence': 0.0, 'context': ''}
    # Prepare context and sources
    context = "\n\n".join([doc [ 'content'] for doc in results ])
    sources = [{
        'source': doc['metadata'].get('source_file', doc['metadata'].get('source', 'unknown')),
        'page': doc['metadata'].get('page', 'unknown'),
        'score': doc['similarity_score'],
        'preview': doc['content'] [:300] + '...'
    } for doc in results]
    confidence = max([doc['similarity_score'] for doc in results])
    
    prompt = f"""Use the following context to answer the question concisely.
                    Context: {context}
                    Question: {query}
                    Answer:
    """
    response = llm.invoke([prompt.format(context=context, query=query)])

    output = {
        'answer': response.content,
        'sources': sources,
        'confidence': confidence
    }

    if return_context:
        output['context'] = context
    return output

result = rag_advanced("Applications of Attention in our Model", rag_retriever, llm, top_k=5, min_score=0.1, return_context=True)
print("Answer: ", result['answer'])
print("Sources: ", result['sources'])
print("Confidence: ", result['confidence'])
print("Context Preview: ", result['context'][:300])


Retrieving documents for query: 'Applications of Attention in our Model'
Top K: 5, Score threshold: 0.1
Generating embeddings for 1 texts...


Batches: 100%|██████████| 1/1 [00:00<00:00, 15.08it/s]

Generated embeddings with shape: (1, 384)
Retrieved 2 documents (after filtering).


Answer:  **Applications of Attention in Our Model**

- **Parallel Multi‑Head Processing**  
  - Queries, keys, and values are linearly projected into \(d_k\), \(d_k\), and \(d_v\) dimensions for each head.  
  - Each head computes attention independently, enabling the model to capture diverse patterns in parallel.

- **Joint Subspace Attention**  
  - By attending to different representation subspaces simultaneously, the model can focus on multiple aspects of the input (e.g., syntax, semantics) at the same time.

- **Concatenation & Final Projection**  
  - The \(d_v\)-dimensional outputs from all heads are concatenated and projected back to the original dimension, producing a rich, fused representation.

- **Enhanced Expressiveness**  
  - Multi‑head attention mitigates the averaging effect of a single head, allowing the network to learn more nuanced relationships across positions.

- **Scalable Context Capture**  
  - The dot‑product attention mechanism scales with the dimensionality